In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [2]:
#laod dataset yang akan digunakan untuk test
df = pd.read_csv("data/bank-additional-full.csv")
df.pop("y")

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'age': 30,
 'job': 'unemployed',
 'marital': 'married',
 'education': 'high.school',
 'default': 'no',
 'housing': 'no',
 'loan': 'no',
 'contact': 'telephone',
 'month': 'may',
 'day_of_week': 'mon',
 'duration': 38,
 'campaign': 1,
 'pdays': 999,
 'previous': 0,
 'poutcome': 'nonexistent',
 'emp.var.rate': 1.1,
 'cons.price.idx': 93.994,
 'cons.conf.idx': -36.4,
 'euribor3m': 4.857,
 'nr.employed': 5191.0}

In [3]:
for kolom in df.columns:
    nilai_unik = df[kolom].nunique()
    print(f"Kolom '{kolom}' memiliki nilai unik: {nilai_unik}")

Kolom 'age' memiliki nilai unik: 78
Kolom 'job' memiliki nilai unik: 12
Kolom 'marital' memiliki nilai unik: 4
Kolom 'education' memiliki nilai unik: 8
Kolom 'default' memiliki nilai unik: 3
Kolom 'housing' memiliki nilai unik: 3
Kolom 'loan' memiliki nilai unik: 3
Kolom 'contact' memiliki nilai unik: 2
Kolom 'month' memiliki nilai unik: 10
Kolom 'day_of_week' memiliki nilai unik: 5
Kolom 'duration' memiliki nilai unik: 1544
Kolom 'campaign' memiliki nilai unik: 42
Kolom 'pdays' memiliki nilai unik: 27
Kolom 'previous' memiliki nilai unik: 8
Kolom 'poutcome' memiliki nilai unik: 3
Kolom 'emp.var.rate' memiliki nilai unik: 10
Kolom 'cons.price.idx' memiliki nilai unik: 26
Kolom 'cons.conf.idx' memiliki nilai unik: 26
Kolom 'euribor3m' memiliki nilai unik: 316
Kolom 'nr.employed' memiliki nilai unik: 11


In [4]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [5]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
dtype: object

In [6]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [7]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://churn-prediction-production.up.railway.app/v1/models/churn-prediction:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json().get("predictions")
    if prediction:
        prediction_value = prediction[0][0]
        result = "No Churn" if prediction_value < 0.5 else "Churn"
    else:
        result = "Error: No predictions found in response."
    print(result)

In [8]:
make_predictions(inputs)

No Churn
